In [1]:
import pandas as pd
import numpy as np
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
# 전처리 된 데이터 불러오기
df = pd.read_csv('/Users/j/Desktop/projects/Project01/gangnam_restaurant.csv')

In [3]:
df['kakao_keyword'] = "강남" + " "+ df['name']
df

,Unnamed: 0,name,state,adress,address_num,type,scale,kakao_keyword
0,0,배초향,영업/정상,서울특별시 강남구 도산대로55길 20 (청담동),135954,한식,0.00,강남 배초향
1,7,브래서리,영업/정상,서울특별시 강남구 봉은사로 524 1층 (삼성동 인터컨티넨탈 서울 코엑스),135090,경양식,950.66,강남 브래서리
2,8,로비라운지,영업/정상,서울특별시 강남구 봉은사로 524 1층 (삼성동 인터컨티넨탈 서울 코엑스),135090,경양식,473.24,강남 로비라운지
3,10,로비바,영업/정상,서울특별시 강남구 봉은사로 524 지상1층 (삼성동 코엑스인터컨티넨탈서울),135090,경양식,282.59,강남 로비바
4,15,노블발렌티,영업/정상,서울특별시 강남구 봉은사로 637 (삼성동),135873,한식,1157.90,강남 노블발렌티
...,...,...,...,...,...,...,...,...
889,14099,샤방샤방,영업/정상,서울특별시 강남구 언주로 405 (역삼동),135921,한식,35.00,강남 샤방샤방
890,14105,젠틀커피,영업/정상,서울특별시 강남구 논현로30길 41 (도곡동),135854,한식,91.40,강남 젠틀커피
891,14121,후포항,영업/정상,서울특별시 강남구 논현로38길 38-3 (도곡동),135854,한식,87.82,강남 후포항
892,14143,유연정,영업/정상,서울특별시 강남구 개포로109길 34 (개포동),135939,한식,38.36,강남 유연정


# xpath 정리

- 여러 곳의 xpath를 찍어보고 다른 부분이 있으면 그 부분만 지우기

## url 0
//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1] <br>
//*[@id="info.search.place.list"]/li[2]/div[5]/div[4]/a[1] <br>
//*[@id="mArticle"]/div[5]/div[3]/div/a[1] <br>
//*[@id="mArticle"]/div[5]/div[3]/div/a[2] <br>


## 타이틀 ㅇ
//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2 <br>

## 별점
//*[@id="mArticle"]/div[5]/div[2]/div/em <br>
//*[@id="mArticle"]/div[5]/div[2]/div/em <br>
//*[@id="mArticle"]/div/div[2]/div/em <br>
//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span/text()

## 리뷰수
 //*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]/text()
 //*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]/text()
//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]
//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]/text()

## 주소 ㅇ
//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1] <br>

## 번지 0
//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[2] <br>

## 리뷰
\
//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span
//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span
//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span
//*[@id="mArticle"]/div[5]/div[3]/ul/li[2]/div[2]/p/span
//*[@id="mArticle"]/div[5]/div[3]/ul/li[3]/div[2]/p/span

In [4]:
driver = webdriver.Chrome('/Users/j/Desktop/projects/Project01/chromedriver')

restaurant_list = []

# df['kakao keyword'] 하나씩 검색
for i, keyword in enumerate(df['kakao_keyword'].tolist()):
    print(keyword, end=" ")

    kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
    driver.get(kakao_map_search_url)
    time.sleep(1)
    
    # 카카오맵의 '상세보기' 클릭해서 들어가기
    try:
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')
        url = url.get_attribute('href')
    except:
        continue
        time.sleep(random.uniform(2,4))

    tmp_list = []

    driver.get(url)
    driver.implicitly_wait(10)

    driver.execute_script("window.scrollTo(0, 20000000)") 
    driver.implicitly_wait(10)
    time.sleep(1)


    # 식당이름
    title = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
    tmp_list.append(title)


    # 주소
    try:
        address = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
        tmp_list.append(address)
    except:
        tmp_list.append("None")


    # 번지수
    try:
        address_num = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[2]').text
        tmp_list.append(address_num)
    except:
        tmp_list.append("None")
        
    
    # 카테고리
    cat = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]/text()').text
    tmp_list.append(cat)
    
    # 메뉴
    menu = driver.find_element_by_xpath('//*[@id="mArticle"]/div[3]/ul/li[1]/div/span').text
    tmp_list.append(menu)

    # 별점 : 별점이 존재하지 않는 곳도 있어 "none" 추가
    try:
        stars = driver.find_element_by_xpath('//*[@id="mArticle"]/div/div[2]/div/em').text
        tmp_list.append(stars)
    except:
        tmp_list.append("None")


    # 리뷰 수
    try:
        review_count = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]').text
        tmp_list.append(review_count)
    except:
        tmp_list.append("None")


    # 리뷰
    review_list = driver.find_elements_by_xpath('//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span')
    tmp_list.append([i.text for i in review_list])
    
    for button_num in range(2, 6):
        # 리뷰 2 ~ 5페이지 클릭 후 크롤링
        try:
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)

            reviews =  driver.find_elements_by_xpath('//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span')
            tmp_list.append([i.text for i in reviews])
            
        except:
            break    

    restaurant_list.append(tmp_list)
    
            
driver.quit()

# 마지막에 numpy 변환 후 2차원으로 reshape 필요
# 주차장 등 음식점이 아닌 곳도 있어 전처리 필요

/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_3082/696044795.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/j/Desktop/projects/Project01/chromedriver')


강남 배초향 

/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_3082/696044795.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')


강남 브래서리 강남 로비라운지 

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=101.0.4951.64)
Stacktrace:
0   chromedriver                        0x00000001028692c9 chromedriver + 5120713
1   chromedriver                        0x00000001027f7e33 chromedriver + 4656691
2   chromedriver                        0x00000001023e7158 chromedriver + 393560
3   chromedriver                        0x00000001023d63a7 chromedriver + 324519
4   chromedriver                        0x00000001023d7822 chromedriver + 329762
5   chromedriver                        0x00000001023d030c chromedriver + 299788
6   chromedriver                        0x00000001023e85b2 chromedriver + 398770
7   chromedriver                        0x000000010244b5fb chromedriver + 804347
8   chromedriver                        0x0000000102439373 chromedriver + 729971
9   chromedriver                        0x000000010240f609 chromedriver + 558601
10  chromedriver                        0x0000000102410635 chro

In [62]:
# '역'이라는 식당 존재로 크롤링에 오류
df[df['name'] == '역']

,Unnamed: 0,name,state,adress,address_num,type,scale
237,4761,역,영업/정상,서울특별시 강남구 언주로134길 6 (논현동),135821,일식,76.0


In [68]:
# 237열에 해당되는 식당 이름이 '역'이기 때문에 238열부터 다시 크롤링

driver = webdriver.Chrome('/Users/j/Desktop/projects/Project01/chromedriver')

restaurant_list2 = []

for i, keyword in enumerate(df['kakao_keyword'][238:].tolist()):
    print(keyword, end=" ")

    kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
    driver.get(kakao_map_search_url)
    time.sleep(1)
    
    try:
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')
        url = url.get_attribute('href')
    except:
        continue
        time.sleep(random.uniform(2,4))

    tmp_list = []

    driver.get(url)
    driver.implicitly_wait(10)

    driver.execute_script("window.scrollTo(0, 20000000)") 
    driver.implicitly_wait(10)
    time.sleep(1)


    # 식당이름
    title = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
    tmp_list.append(title)


    # 주소
    try:
        address = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
        tmp_list.append(address)
    except:
        tmp_list.append("None")


    # 번지수
    try:
        address_num = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[2]').text
        tmp_list.append(address_num)
    except:
        tmp_list.append("None")


    # 별점 : 별점이 존재하지 않는 곳도 있어 "none" 추가
    try:
        stars = driver.find_element_by_xpath('//*[@id="mArticle"]/div/div[2]/div/em').text
        tmp_list.append(stars)
    except:
        tmp_list.append("None")


    # 리뷰 수
    try:
        review_count = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[2]').text
        tmp_list.append(review_count)
    except:
        tmp_list.append("None")


    # 리뷰(일단 첫 페이지)
    review_list = driver.find_elements_by_xpath('//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span')
    tmp_list.append([i.text for i in review_list])
    
    for button_num in range(2, 6):
        # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
        try:
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)

            reviews =  driver.find_elements_by_xpath('//*[@id="mArticle"]/div[5]/div[3]/ul/li/div[2]/p/span')
            tmp_list.append([i.text for i in reviews])
            
        except:
            break    

    restaurant_list2.append(tmp_list)
    
            
driver.quit()

/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/j/Desktop/projects/Project01/chromedriver')


강남 (신)오양절면 

/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')


강남 굴마을굴국밥 

/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  title = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:36: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  address = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:44: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  address_num = driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[2]').text
/var/folders/h_/vntlnhwx04ldxdcw4k9ftgkh0000gn/T/ipykernel_6967/1847555934.py:52: DeprecationWarning: find

강남 강성환차이나 강남 계몽돈까스 강남 엄마정원 강남 (주)소노인터내셔널 삼성지점 강남 이지상사 카메오 강남 곱창고 강남 별들의 기침 강남 순천만 강남 베스트 강남 대범천 강남 난랑 강남 완산골명가 강남구청역점 강남 프로간장게장 강남 스타닭갈비 강남 하모 강남 골목길 강남 마포네 강남 요기당 강남 다다돈까스 강남 강남웨딩 출장부페 강남 수정방 강남 오늘 와인한잔 세로수길점 강남 압구정곱떡 가로수길점 강남 누락된 업소 강남 연남토마(강남점) 강남 청기와생고기 강남 언양불고기 강남 현대가 한우 소머리국밥 강남 새마을식당 대치점 강남 한방정통삼계탕 강남 샤브리스 강남 제메이 양꼬치와 샤브샤브 강남 파파치킨 강남 영양센타대치점 강남 마왕족발 강남 동호치킨 강남 허수아비돈까스 강남 텐텐 강남 향토통닭구이 강남 영양센타압구정점 강남 김북순큰남비집 강남 밤푸스치킨 강남 (주)김돈이 강남 (주)탐나종합어시장신사역점 강남 세자매복집 강남 (주)예 강남 삼호짱뚱이 강남 나따오비까(대치점) 강남 삼삼이네 강남 미인 이자카야포차 강남 뉴델리 강남 피자노베 강남 동강 강남 두메 강남 취향의공간 강남 남산희래등 강남 신대흥장 강남 장성생태전문점 강남 장군숯불갈비 강남 시나브로 강남 도곡원 강남 장독대 강남 한우물 강남 가야지 양평해장국 강남 한성양꼬치 강남 종로화덕 강남 전주순대 강남 사또족발 강남 오시오 강남 제주탐하리 강남 영덕막회 강남 라무다이닝 강남 이소 강남 혜정식당 강남 신촌설렁탕(역삼점) 강남 야다이 강남 거자필반 강남 아띠랑스 강남 뽕사부 강남 선릉 순대집 강남 청담이상 본점 강남 진미 평양냉면 강남 토영자갈치곰장어 강남 마요네즈 신논현점 강남 서초동연가 강남 충무상회 강남 돈부자 강남 남해굴국밥 강남 영동소금구이 강남 남도애찬 강남 전주콩나물국밥 강남 노란상 소갈비 강남직영점 강남 예당어죽민물매운탕 강남 주식회사 안동한우원 강남 춘하추동밀면 대치직영점 강남 어방참치 강남 옛날농장 강남 청숫골 강남 정가네부엌 강남 김가네 강남 오울 강남 장독묵은지 강남 하나샤브정 강남 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.64)
Stacktrace:
0   chromedriver                        0x00000001067842c9 chromedriver + 5120713
1   chromedriver                        0x0000000106712e33 chromedriver + 4656691
2   chromedriver                        0x0000000106302158 chromedriver + 393560
3   chromedriver                        0x00000001062e24c8 chromedriver + 263368
4   chromedriver                        0x00000001063579bb chromedriver + 743867
5   chromedriver                        0x00000001063665af chromedriver + 804271
6   chromedriver                        0x0000000106354373 chromedriver + 729971
7   chromedriver                        0x000000010632a609 chromedriver + 558601
8   chromedriver                        0x000000010632b635 chromedriver + 562741
9   chromedriver                        0x0000000106756b1d chromedriver + 4934429
10  chromedriver    

In [70]:
len(restaurant_list2)

265